In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
import faiss  
import pickle
import umap
from sklearn.metrics import davies_bouldin_score, calinski_harabasz_score, silhouette_score
from sklearn.preprocessing import normalize
from itertools import product 
import random
from joblib import Parallel, delayed
import hdbscan

In [2]:
np.random.seed(42)  
sns.set_theme(style="white", palette="muted")

In [3]:
def load_data():
    books_list = []

    with open('../Pickle/books.pkl', 'rb') as file:
        while True:
            try:
                chunk = pickle.load(file)
                books_list.append(chunk)
            except EOFError:
                break  
    books = pd.concat(books_list, ignore_index=True)
    books = books.drop_duplicates(subset='title', keep='first')
    embedding_matrix = np.vstack(books['embeddings'].values)
    return books, embedding_matrix

In [4]:
def apply_umap(embeddings, n_components=10, n_neighbors=300, min_dist=0.0):
    embeddings = np.asarray(embeddings, dtype=np.float32)  

    umap_model = umap.UMAP(
        n_components=n_components,
        n_neighbors=n_neighbors,
        min_dist=min_dist,
        metric='cosine',
        low_memory=True, 
        random_state = 42
    )
    
    return umap_model.fit_transform(embeddings)

In [5]:
def assign_clusters_to_books(books, soft_clusters, embeddings, top_n=5):
    """
    Return a DataFrame with only book_id, embeddings, and top N cluster memberships.
    
    Parameters:
    - books: Original books DataFrame. Only the 'book_id' column is used here.
    - soft_clusters: Soft cluster membership vectors (probabilities for each cluster).
    - embeddings: The UMAP normalized embeddings for each book.
    - top_n: Number of top clusters to keep.
    
    Returns:
    - clustered_books: A simplified DataFrame with book_id, embedding, and top_clusters.
    """
    
    # Convert soft_clusters to a NumPy array if not already
    if not isinstance(soft_clusters, np.ndarray):
        soft_clusters = np.array(soft_clusters)

    # Get the top N clusters and their probabilities for each book
    top_clusters_list = []
    for cluster_vector in soft_clusters:
        top_indices = np.argsort(cluster_vector)[::-1][:top_n]
        top_probs = cluster_vector[top_indices]
        top_clusters = list(zip(top_indices, top_probs))
        top_clusters_list.append(top_clusters)
    
    # Create a new DataFrame with just the needed columns
    clustered_books = pd.DataFrame({
        'book_id': books['book_id'].values,
        'embedding': [embedding.tolist() for embedding in embeddings],  # Convert arrays to lists
        'top_clusters': top_clusters_list
    })
    
    return clustered_books

In [ ]:
def perform_hdbscan_clustering(embeddings, alpha=0.5, beta=0.5, n_trials=5, n_jobs=-1):
    """
    Perform HDBSCAN clustering on the given embeddings using soft clustering (probabilistic membership vectors).
    This function avoids precomputing the distance matrix to save memory.

    Args:
        embeddings (numpy.ndarray): The normalized embeddings (e.g., UMAP embeddings) of the books or items to cluster.
        alpha (float, optional): Weight for the Davies-Bouldin Index in the combined score. Default is 0.5.
        beta (float, optional): Weight for the Calinski-Harabasz Index in the combined score. Default is 0.5.
        n_trials (int, optional): The number of random hyperparameter combinations to try. Default is 5.
        n_jobs (int, optional): The number of jobs to run in parallel. Default is -1 (all cores).

    Returns:
        tuple: A tuple containing:
            - best_soft_clusters (numpy.ndarray): The best soft clusters, each element contains the membership 
              probabilities for each cluster.
            - best_hard_clusters (numpy.ndarray): The best hard cluster labels (assigned clusters).
            - best_clusterer (hdbscan.HDBSCAN): The best fitted HDBSCAN model used to generate the soft clusters.
            - best_db_score (float): The Davies-Bouldin score for the best clustering.
            - best_ch_score (float): The Calinski-Harabasz score for the best clustering.
            - best_combined_score (float): The combined score for the best clustering.
            - best_params (dict): The hyperparameters that gave the best score.
    """
    # Define hyperparameter search space
    min_cluster_sizes = [100, 90, 80]
    min_samples_list = [50, 60, 70]
    cluster_selection_epsilons = [0.1, 0.5]

    # Generate all possible hyperparameter combinations
    all_param_combinations = list(product(min_cluster_sizes, min_samples_list, cluster_selection_epsilons))

    # Sample n_trials parameter combinations
    sampled_combinations = random.sample(all_param_combinations, min(n_trials, len(all_param_combinations)))

    # Function to evaluate a single hyperparameter combination
    def evaluate_params(min_cluster_size, min_samples, cluster_selection_epsilon):
        clusterer = hdbscan.HDBSCAN(
            min_cluster_size=min_cluster_size,
            min_samples=min_samples,
            cluster_selection_epsilon=cluster_selection_epsilon,
            metric='euclidean', 
            prediction_data=True,
            core_dist_n_jobs=n_jobs,  # Use multiple cores for distance computations
            cluster_selection_method='leaf'
        )
        clusterer.fit(embeddings)

        # Get the soft clusters (membership vectors)
        soft_clusters = hdbscan.prediction.all_points_membership_vectors(clusterer)

        # Compute clustering scores based on hard clusters (labels_)
        if len(soft_clusters) > 0:
            db_index = davies_bouldin_score(embeddings, clusterer.labels_)
            ch_index = calinski_harabasz_score(embeddings, clusterer.labels_)
        else:
            db_index, ch_index = float("inf"), 0

        # Calculate combined score
        combined_score = alpha * (1 / db_index) + beta * ch_index

        # Print the parameter combination and scores
        print(f"min_cluster_size={min_cluster_size}, min_samples={min_samples}, epsilon={cluster_selection_epsilon}, DB={db_index:.3f}, CH={ch_index:.3f}, Combined={combined_score:.3f}")

        return combined_score, db_index, ch_index, soft_clusters, clusterer.labels_, clusterer, (min_cluster_size, min_samples, cluster_selection_epsilon)

    # Evaluate all parameter combinations in parallel
    results = Parallel(n_jobs=n_jobs)(
        delayed(evaluate_params)(min_cluster_size, min_samples, cluster_selection_epsilon)
        for min_cluster_size, min_samples, cluster_selection_epsilon in sampled_combinations
    )

    # Find the best result
    best_index = np.argmax([result[0] for result in results])
    best_combined_score, best_db_score, best_ch_score, best_soft_clusters, best_hard_clusters, best_clusterer, best_params = results[best_index]

    # Print the best hyperparameters and scores
    print("\nBest Hyperparameters:")
    print(f"min_cluster_size={best_params[0]}, min_samples={best_params[1]}, epsilon={best_params[2]}")
    print("Best Combined Score:", best_combined_score)
    print("Best Davies-Bouldin Score:", best_db_score)
    print("Best Calinski-Harabasz Score:", best_ch_score)

    return best_soft_clusters, best_hard_clusters, best_clusterer, best_db_score, best_ch_score, best_combined_score, best_params

In [7]:
books, embedding_matrix = load_data()
scaler = StandardScaler()
scaled_embeddings = scaler.fit_transform(embedding_matrix)

In [8]:
# umap_embeddings = apply_umap(scaled_embeddings)
umap_embeddings = pd.read_pickle('../Pickle/umap_embeddings.pkl')

In [9]:
#umap_embeddings_normalized = normalize(umap_embeddings, norm='l2', axis=1)

In [10]:
best_soft_clusters, best_hard_clusters, best_clusterer, best_db_score, best_ch_score, best_combined_score, best_params = perform_hdbscan_clustering(umap_embeddings)


Best Hyperparameters:
min_cluster_size=100, min_samples=50, epsilon=0.5
Best Combined Score: 647.888597436619
Best Davies-Bouldin Score: 0.7294188426377485
Best Calinski-Harabasz Score: 1294.4062404345182


In [11]:
indices = np.arange(umap_embeddings.shape[0])

In [12]:
clustered_books = assign_clusters_to_books(books, best_soft_clusters, umap_embeddings, top_n=5)

In [13]:
book_id_to_index = {book_id: idx for idx, book_id in enumerate(books['book_id'])}

In [14]:
dimension = umap_embeddings.shape[1]
faiss_index = faiss.IndexFlatL2(dimension)
faiss_index.add(umap_embeddings)
with open('../Pickle/umap_embeddings.pkl', 'wb') as f:
    pickle.dump(umap_embeddings, f)
faiss.write_index(faiss_index, '../Pickle/faiss_index.bin')
with open('../Pickle/book_id_to_index.pkl', 'wb') as f:
    pickle.dump(book_id_to_index, f)


In [15]:
clustered_books

,book_id,embedding,top_clusters
0,1,"[0.4171278774738312, 0.10337015986442566, 0.30...","[(32, 1.0), (82, 3.516317889250751e-305), (190..."
1,27,"[0.3890656530857086, 0.08596204966306686, 0.30...","[(199, 0.02683962260000989), (116, 0.005909768..."
2,40,"[0.3129916787147522, 0.0636841356754303, 0.224...","[(175, 0.05767255857016169), (174, 0.032773852..."
3,45,"[0.2904050052165985, 0.14248579740524292, 0.32...","[(83, 0.07605937165158964), (223, 0.0084628273..."
4,48,"[0.3875168561935425, 0.019565841183066368, 0.3...","[(58, 0.00022819556134684034), (30, 0.00017583..."
...,...,...,...
454493,36483546,"[0.396977961063385, 0.06940219551324844, 0.357...","[(214, 0.029483059832237285), (204, 0.01112924..."
454494,36488099,"[0.41653749346733093, -0.05733920633792877, 0....","[(244, 0.050482147865868986), (248, 0.00603390..."
454495,36491811,"[0.38446611166000366, -0.04149968549609184, 0....","[(263, 0.0036537117744564547), (237, 0.0013087..."
454496,36494299,"[0.4507542848587036, 0.21118517220020294, 0.27...","[(4, 0.7674472088737923), (16, 0.0022876522288..."


In [19]:
with open('../Pickle/clustered_books.pkl', 'wb') as f:
    pickle.dump(clustered_books, f)

In [18]:
from sklearn.metrics.pairwise import euclidean_distances

compactness = []
for label in set(best_hard_clusters):
    if label != -1:
        cluster_points = umap_embeddings[best_hard_clusters == label]
        centroid = cluster_points.mean(axis=0)
        distances = euclidean_distances(cluster_points, centroid.reshape(1, -1))
        compactness.append(np.mean(distances))

separation = []
cluster_centroids = [
    umap_embeddings[best_hard_clusters == label].mean(axis=0)
    for label in set(best_hard_clusters) if label != -1
]

for i in range(len(cluster_centroids)):
    for j in range(i + 1, len(cluster_centroids)):
        dist = euclidean_distances(
            [cluster_centroids[i]], [cluster_centroids[j]]
        )[0][0]
        separation.append(dist)

print(f"Average Compactness: {np.mean(compactness):.4f}")
print(f"Average Separation: {np.mean(separation):.4f}")

total_points = len(best_hard_clusters)
outlier_points = np.sum(best_hard_clusters == -1)
outlier_percentage = (outlier_points / total_points) * 100
print(f"Outliers: {outlier_points} / {total_points}")
print(f"Percentage of Outliers: {outlier_percentage:.2f}%")
dbi_score = davies_bouldin_score(umap_embeddings, best_hard_clusters)
print(f"Davies-Bouldin Index: {dbi_score}")
ch_score = calinski_harabasz_score(umap_embeddings, best_hard_clusters)
print(f"Calinski-Harabasz Index: {ch_score}")
sh = silhouette_score(umap_embeddings, best_hard_clusters)
print(f"Silhouette Score: {sh}")

Average Compactness: 0.0514
Average Separation: 0.1408
Outliers: 0 / 454498
Percentage of Outliers: 0.00%
Davies-Bouldin Index: 0.7294188426377485
Calinski-Harabasz Index: 1294.4062404345182


KeyboardInterrupt: 

In [ ]:
import plotly.express as px
umap_embeddings_2d = apply_umap(scaled_embeddings, n_components=2)

In [ ]:
filtered_indices = best_hard_clusters != -1
filtered_embeddings_2d = umap_embeddings_2d[filtered_indices]
filtered_clusters = best_hard_clusters[filtered_indices]
df_plot_2d = pd.DataFrame({
    'UMAP1': filtered_embeddings_2d[:, 0],
    'UMAP2': filtered_embeddings_2d[:, 1],
    'Cluster': filtered_clusters
})
fig = px.scatter(
    df_plot_2d,
    x='UMAP1',
    y='UMAP2',
    color='Cluster',
    title='2D UMAP Embeddings Coloured by Cluster',
    opacity=0.7
)

fig.update_layout(
    xaxis_title='UMAP Dimension 1',
    yaxis_title='UMAP Dimension 2'
)

fig.show()

In [ ]:
plt.figure(figsize=(10, 8))
sns.scatterplot(
    x=filtered_embeddings_2d[:, 0],
    y=filtered_embeddings_2d[:, 1],
    hue=filtered_clusters,
    palette='tab10',
    alpha=0.7,
    legend = False
)
plt.title('2D Plot of Embeddings and Clusters', fontsize=16)
plt.xlabel('UMAP Dimension 1')
plt.ylabel('UMAP Dimension 2')
plt.show()


In [ ]:
plt.figure(figsize=(10, 8))
sns.kdeplot(
    x=filtered_embeddings_2d[:, 0],
    y=filtered_embeddings_2d[:, 1],
    fill=True,
    cmap='viridis',  
    alpha=0.5
)

plt.title('Density Plot of UMAP 2D Embeddings')
plt.xlabel('UMAP Dimension 1')
plt.ylabel('UMAP Dimension 2')
plt.show()


In [ ]:
plt.figure(figsize=(10, 8))
sns.kdeplot(
    x=filtered_embeddings_2d[:, 0],
    y=filtered_embeddings_2d[:, 1],
    hue=filtered_clusters,
    fill=True,
    alpha=0.5,
    palette='tab10',
    legend=False 
)

plt.title('Density Plot of UMAP 2D Embeddings by Cluster')
plt.xlabel('UMAP Dimension 1')
plt.ylabel('UMAP Dimension 2')
plt.show()


In [ ]:
unique, counts = np.unique(filtered_clusters, return_counts=True)
cluster_sizes = dict(zip(unique, counts))
df = pd.DataFrame({
    'Cluster': list(cluster_sizes.keys()),
    'Count': list(cluster_sizes.values())
})
plt.figure(figsize=(50, 6))
sns.barplot(data=df, x='Cluster', y='Count', hue='Cluster', palette='viridis', dodge=False, legend=False)
plt.title('Cluster Sizes (With Outliers)', fontsize=16)
plt.xlabel('Cluster Label', fontsize=14)
plt.ylabel('Number of Points', fontsize=14)
plt.xticks(rotation=0, fontsize=12)
plt.yticks(fontsize=12)
plt.tight_layout()
plt.show()

In [ ]:
cluster_sizes = [np.sum(best_hard_clusters == label) for label in set(best_hard_clusters) if label != -1]
plt.hist(cluster_sizes, bins=10, color='skyblue', edgecolor='black')
plt.title('Cluster Size Distribution')
plt.xlabel('Cluster Size')
plt.ylabel('Frequency')
plt.show()